In [1]:
# 무한 스크롤 부분에 try except로 key down이 안되는 경우(페이지 맨 밑 바닥인 경우) while 반복문 취소하게 했었다. 
# 그런데 이게 except 처리가 안 되더라. 그냥 건너뛰어서 li 갯수가 적은 경우(예:24개) key page_down에서 걸려서 무한반복하고 있더라. 
# 이 오류 수정했다. 
# headless 실행하니까 메모리 문제인지 크롤링 하다가 중간에 종료된다. 그래서 원하는 갯수만큼 충분히 데이터 수집이 안 된다. 없앴다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt
import pyautogui
from selenium.webdriver.common.keys import Keys

import random
import os  

from selenium.webdriver.support.ui import Select


#크롤링 기본 셋팅
print("=" *80)
print("연습문제 6-5 : 블로그 크롤러 : 여러건의 네이버 블로그 정보 추출하여 저장하기")
print("=" *80)

key_word = input('''1. 크롤링할 키워드는 무엇입니까?(예:여행):''')

include_word = input('''2. 결과에서 반드시 포함하는 단어를 입력하세요(예:국내, 바닷가)
(여러개일 경우 , 로 구분해서 입력하고 없으면 엔터 입력하세요): ''')

exclude_word = input('''3. 결과에서 제외할 단어를 입력하세요(예:분양권, 해외)
(여러개일 경우 , 로 구분해서 입력하고 없으면 엔터 입력하세요):''')

start_date = input('4. 조회 시작일자 입력(예:2017-01-01):')

end_date = input('5. 조회 종료일자 입력(예:2017-12-31):')

cnt = int(input('6. 크롤링 할 건수는 몇건입니까?(최대 1000건): '))


f_dir = input("7. 파일을 저장할 폴더명만 쓰세요(예:/Users/kibeomkim/Desktop/folder1/): ")
if f_dir=='' :
    f_dir='/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'
    
    
# 저장될 파일위치와 이름 지정하기
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+key_word)
os.chdir(f_dir+s+'-'+key_word)

ff_name= s+'-'+key_word+'.txt'
fc_name= s+'-'+key_word+'.csv'
fx_name= s+'-'+key_word+'.xls'

# 크롬 드라이버를 사용해서 웹 브라우저 실행하기 

s_time = time.time( )

#options = webdriver.ChromeOptions()
#options.add_argument('headless')
#options.add_argument('lang=ko_KR')
#options.add_argument('disable-gpu')
#options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')


path = "/Users/kibeomkim/Desktop/chromedriver_240/chromedriver"
driver = webdriver.Chrome(path)

#1페이지로 이동
driver.get('https://www.naver.com/')
time.sleep(random.randrange(2,5))


#네이버 검색창 찾고 검색어 넣고 엔터
element1 = driver.find_element_by_id('query')
element1.send_keys(key_word)
element1.send_keys('\n')
time.sleep(2)



연습문제 6-5 : 블로그 크롤러 : 여러건의 네이버 블로그 정보 추출하여 저장하기
1. 크롤링할 키워드는 무엇입니까?(예:여행):다꾸
2. 결과에서 반드시 포함하는 단어를 입력하세요(예:국내, 바닷가)
(여러개일 경우 , 로 구분해서 입력하고 없으면 엔터 입력하세요): 
3. 결과에서 제외할 단어를 입력하세요(예:분양권, 해외)
(여러개일 경우 , 로 구분해서 입력하고 없으면 엔터 입력하세요):
4. 조회 시작일자 입력(예:2017-01-01):2020.01.01
5. 조회 종료일자 입력(예:2017-12-31):2020.12.31
6. 크롤링 할 건수는 몇건입니까?(최대 1000건): 10
7. 파일을 저장할 폴더명만 쓰세요(예:/Users/kibeomkim/Desktop/folder1/): 


In [2]:
#검색옵션 선택하기 
try : 
    driver.find_element_by_class_name('bt_option').click()
    time.sleep(2)
except : 
    print("")

driver.find_element_by_link_text('기간').click()
time.sleep(2)
element2 = driver.find_element_by_id('_nx_date_from')
element2.send_keys(start_date)
element3 = driver.find_element_by_id('_nx_date_to')
element3.send_keys(end_date)

driver.find_element_by_class_name('tx').click()
time.sleep(2)

driver.find_element_by_link_text('상세검색').click()
time.sleep(2)

element4 = driver.find_element_by_class_name('_input_include_')
element4.send_keys(include_word)

element5 = driver.find_element_by_class_name('_input_exclude_')
element5.send_keys(exclude_word)

driver.find_element_by_link_text('검색').click()
time.sleep(2)

driver.find_element_by_link_text('블로그').click()
time.sleep(2)

driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div/ul/li[5]/a').click()
time.sleep(2)
driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div/ul/li[5]/div/div/div[1]/div[1]/input').click()
time.sleep(0.5)
text_search = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div/ul/li[5]/div/div/div[2]/input')
time.sleep(0.5)
text_search.send_keys('blog.naver.com/')
driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div/ul/li[5]/div/div/span/button/span').click()
time.sleep(2)


title_list1 = []
url_list1 = []
nick_list1 = []
date_list1 = []
con_list1 = []

no2 = 2
#무한스크롤링 


while 1 == 1 : 
    html2 = driver.page_source
    soup2 = BeautifulSoup(html2, 'html.parser')
    blog_list011 = soup2.find('ul', 'lst_total').find_all('li') 
    old = len(blog_list011)
    if cnt > len(blog_list011):
        body = driver.find_element_by_css_selector('body')
        for x in range(1, 8) : 
            body.send_keys(Keys.PAGE_DOWN)    
            time.sleep(2)
        html3 = driver.page_source
        time.sleep(2)
        soup3 = BeautifulSoup(html3, 'html.parser')
        blog_list022 = soup3.find('ul', 'lst_total').find_all('li')
        if old == len(blog_list022) : 
            print('%s건 을 요청하셨으나, 게시글이 %s 건 밖에 존재하지 않아 %s 건 만큼만 수집하겠습니다.' %(cnt, len(blog_list022), len(blog_list022)))
            cnt = len(blog_list022)
            break
    elif cnt == len(blog_list011) : 
        break
    elif cnt < len(blog_list011) : 
        break

In [3]:

contents_list111 = ""
blog_url_list222 = []
html12 = driver.page_source
soup12 = BeautifulSoup(html12, 'html.parser')
contents_list12 = soup12.find('ul', 'lst_total').find_all('li', 'bx')
no3 = 1

for b in contents_list12 : 
    blog_url12 = b.find('a', 'api_txt_lines total_tit')['href']
    blog_url_list222.append(blog_url12)
    no3 +=1
    if no3 > cnt : 
        break
driver.quit()

no1 = 1
f = open(ff_name, 'a', encoding='utf-8')
for c in blog_url_list222 : 
    #options = webdriver.ChromeOptions()
    #options.add_argument('headless')
    #options.add_argument('lang=ko_KR')
    #options.add_argument('disable-gpu')
    #options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

    path = "/Users/kibeomkim/Desktop/chromedriver_240/chromedriver"
    driver = webdriver.Chrome(path)
    driver.get(c)
    time.sleep(5)
    
    html0 = driver.page_source
    soup0 = BeautifulSoup(html0, 'html.parser')
    try : 
        asdf = driver.switch_to.frame('mainFrame')
        html900 = driver.page_source
        soup900 = BeautifulSoup(html900, 'html.parser')
        print('총 %s 건 중 %s 번째 블로그 데이터를 수집하는 중입니다====================='%(len(blog_url_list222), no1))
    
        f.write('총 %s 건 중 %s 번째 블로그 데이터를 수집합니다======================'%(len(blog_url_list222), no1))
        f.write('\n')
    
        f.write('0. 블로그 주소 :'+c)
        f.write('\n')
        url_list1.append(c)
        
        
        try : 
            contents_title = soup900.find('div', 'se-module se-module-text se-title-text').get_text().replace('\n', "").strip()
            f.write('1. 블로그 제목 :'+contents_title)
            f.write('\n')
            title_list1.append(contents_title)
        except : 
            f.write('1. 블로그 제목 :')
            f.write('\n')
            title_list1.append(" ")
        
    
        try : 
            nick_name = soup900.find('a', 'link pcol2').get_text().replace('\n', "").strip()
        except : 
            nick_name = soup900.find('strong', 'itemfont col').get_text().replace('\n', "").replace(' ', "").strip()
        f.write('2. 작성자 :'+nick_name)
        f.write('\n')
        nick_list1.append(nick_name)
    
        try : 
            date_info = soup900.find('span', 'se_publishDate pcol2').get_text().replace('\n', "").strip()
        except : 
            date_info = soup900.find('p', 'date fil5 pcol2 _postAddDate').get_text().replace('/n', "").strip()
        f.write('3. 작성일자 :'+date_info)
        f.write('\n')
        date_list1.append(date_info)
        
        try : 
            contents = soup900.find('div', 'se-main-container').get_text().replace('\n', "").strip()
        except : 
            try : 
                contents = soup900.find('div', id = 'postViewArea').get_text().replace('\n', "").strip()
            except : 
                contents = soup900.find('div', 'se_component_wrap sect_dsc __se_component_area').get_text().replace('\n', "").strip()
        con_list1.append(contents)
        f.write('4. 블로그 내용 :'+contents)
        f.write('\n')
        f.write('\n') 
        
        driver.close()
        no1 += 1
    
    #플러스 알파
    except : 
        html900 = driver.page_source
        soup900 = BeautifulSoup(html900, 'html.parser')
        print('총 %s 건 중 %s 번째 블로그 데이터를 수집하는 중입니다====================='%(len(blog_url_list222), no1))

    
        f.write('총 %s 건 중 %s 번째 블로그 데이터를 수집합니다======================'%(len(blog_url_list222), no1))
        f.write('\n')
    
        f.write('1. 블로그 주소 :'+c)
        f.write('\n')
        url_list1.append(c)
    
        try : 
            nick_name = soup900.find('span', 'author').get_text().replace('\n', "").strip()
        except : 
            nick_name = soup900.find('strong', 'itemfont col').get_text().replace('\n', "").replace(' ', "").strip()
        f.write('2. 작성자 :'+nick_name)
        f.write('\n')
        nick_list1.append(nick_name)
    
        try : 
            date_info = soup900.find('span', 'cB_Tdate').get_text().replace('\n', "").strip()
        except : 
            date_info = soup900.find('p', 'date fil5 pcol2 _postAddDate').get_text().replace('/n', "").strip()
        f.write('3. 작성일자 :'+date_info)
        f.write('\n')
        date_list1.append(date_info)
    
        contents_list2 = soup900.find('div', 'cContentBody').get_text().replace('\n', "").strip()
        con_list1.append(contents_list2)
        f.write('4. 블로그 내용 :'+contents_list2)
        f.write('\n')
        f.write('\n') 
        
        driver.quit()
        no1 += 1
        
    
f.close()

e_time = time.time()
total_time = e_time - s_time

naver = pd.DataFrame()
naver['블로그제목'] = title_list1
naver['블로그주소'] = url_list1
naver['작성자닉네임'] = nick_list1
naver['작성일자'] = date_list1
naver['블로그내용'] = con_list1

naver.to_csv(fc_name, encoding='utf-8-sig')
naver.to_excel(fx_name)

print('='*100)
print('요청하신 %s 건 자료를 모두 수집하였습니다' %cnt)
print('데이터 수집에 걸린 소요시간은 %s 입니다' %total_time)
print('='*100)

총 10 건 중 1 번째 블로그 데이터를 수집하는 중입니다=====================


KeyboardInterrupt: 

In [33]:
len(blog_url_list222)

120